In [1]:
import pandas as pd
import numpy as np
import os

main_dir = os.environ.get("MAIN_DIRECTORY")
os.chdir(main_dir)

from preprocessing.preprocessor import preprocess_audio

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical

In [2]:
model_save_path = os.environ.get("PRETRAINED_MODELS_PATH")

In [3]:
sentiments = ['negative', 'neutral', 'positive']
print(sentiments)

['negative', 'neutral', 'positive']


In [4]:
data = preprocess_audio("models/audio_model/data")

In [5]:
data.head()

,Name,Features,Label
0,03-01-08-02-02-02-23.wav,"[-3.4641016, -3.4641016, -3.4641016, -3.464101...",0
1,03-01-08-02-02-02-23.wav,"[-3.464102, -3.464102, -3.464102, -3.464102, -...",0
2,03-01-08-02-02-02-23.wav,"[-3.4641018, -3.4641018, -3.4641018, -3.464101...",0
3,03-01-08-02-02-02-23.wav,"[-3.4641013, -3.4641013, -3.4641013, -3.464101...",0
4,03-01-08-02-02-02-23.wav,"[-3.4641001, -3.463962, -3.4638703, -3.4640238...",0


In [38]:
model = Sequential([
    Dense(2048, activation='relu', input_shape=(2145,)),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

In [39]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 2048)              4395008   
                                                                 
 dense_31 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_32 (Dense)            (None, 128)               65664     
                                                                 
 dense_33 (Dense)            (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_34 (Dense)            (None, 32)               

In [40]:
opt = Adam(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [41]:
X = np.array(data["Features"].to_list())
y = to_categorical(data['Label'], num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
model.fit(X_train, y_train, epochs=25, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/25
60/60 [==============================] - 2s 29ms/step - loss: 0.9020 - accuracy: 0.5418 - val_loss: 0.8512 - val_accuracy: 0.5875
Epoch 2/25
60/60 [==============================] - 2s 28ms/step - loss: 0.8728 - accuracy: 0.5743 - val_loss: 0.8409 - val_accuracy: 0.5875
Epoch 3/25
60/60 [==============================] - 2s 27ms/step - loss: 0.8424 - accuracy: 0.5805 - val_loss: 0.8416 - val_accuracy: 0.5875
Epoch 4/25
60/60 [==============================] - 2s 27ms/step - loss: 0.8185 - accuracy: 0.5805 - val_loss: 0.8363 - val_accuracy: 0.6000
Epoch 5/25
60/60 [==============================] - 2s 26ms/step - loss: 0.7949 - accuracy: 0.6036 - val_loss: 0.8504 - val_accuracy: 0.5458
Epoch 6/25
60/60 [==============================] - 1s 25ms/step - loss: 0.7625 - accuracy: 0.6287 - val_loss: 0.8671 - val_accuracy: 0.5625
Epoch 7/25
60/60 [==============================] - 2s 26ms/step - loss: 0.7203 - accuracy: 0.6695 - val_loss: 0.8502 - val_accuracy: 0.6167
Epoch 8/25
60

In [11]:
model_json = model.to_json()
with open(os.path.join(model_save_path, "audio_model.json"), "w") as file:
    file.write(model_json)